### Disease Diagnosis

In [1]:
import boto3
import os

def data_ingestion_from_s3():
    s3 = boto3.client(
        's3',
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=os.getenv("AWS_REGION")
    )

    bucket_name = os.getenv("S3_BUCKET_NAME")
    prefix = "carequery/"

    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    local_files = os.getenv("S3_DOWNLOADED_FILES")
    os.makedirs(local_files,exist_ok=True )


    for obj in response.get("Contents", []):
        key = obj["Key"]
        if key.endswith(".csv"):
            filename = os.path.basename(key)
            local_csv_path = os.path.join(local_files,filename)
            if not os.path.exists(local_csv_path):
                s3_obj = s3.get_object(Bucket=bucket_name, Key=key)
                csv_content = s3_obj["Body"].read().decode("utf-8")
                # Write to a temporary file
                with open(local_csv_path,'w', encoding="utf-8") as f:
                    f.write(csv_content)
    return local_files


In [2]:
### Document Structure - DATA Ingestion
import os
from langchain_core.documents import Document
## Directory Loader - To read all files from a directory
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import CSVLoader
import os
## load all the text fies from the directory
csv_files = data_ingestion_from_s3()
dir_loader = DirectoryLoader(
    
    csv_files,
    glob = "**/*.csv", # Pattern to match files
    loader_cls= CSVLoader, # loader class to use
    loader_kwargs={'encoding':'utf-8'},
    show_progress=False
)

documents =dir_loader.load()

print(len(documents))


5135


### Chunking Ingested data

#### 1. make method with parameters documents, chunk_size,chunk_overlap
#### 2. call RecursiveCharacterTextSplitter and give all values to parameters
#### 3. give use split_documents method to split data then print example of chunk



In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_documents(documents,chunk_size=5000,chunk_overlap=500):
    """Split documents into chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        separators=['\n\n',"\n"," ",""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    #Example of chunks
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content:{split_docs[0].page_content[:300]}...")
        print(f"Metadata:{split_docs[0].metadata}")
    return split_docs


In [4]:
chunks = split_documents(documents)
chunks


Split 5135 documents into 5135 chunks

Example chunk:
Content:Disease: Drug Reaction
Precaution_1: stop irritation
Precaution_2: consult nearest hospital
Precaution_3: stop taking drug
Precaution_4: follow up...
Metadata:{'source': 's3_data/symptom_precaution.csv', 'row': 0}


[Document(metadata={'source': 's3_data/symptom_precaution.csv', 'row': 0}, page_content='Disease: Drug Reaction\nPrecaution_1: stop irritation\nPrecaution_2: consult nearest hospital\nPrecaution_3: stop taking drug\nPrecaution_4: follow up'),
 Document(metadata={'source': 's3_data/symptom_precaution.csv', 'row': 1}, page_content='Disease: Malaria\nPrecaution_1: Consult nearest hospital\nPrecaution_2: avoid oily food\nPrecaution_3: avoid non veg food\nPrecaution_4: keep mosquitos out'),
 Document(metadata={'source': 's3_data/symptom_precaution.csv', 'row': 2}, page_content='Disease: Allergy\nPrecaution_1: apply calamine\nPrecaution_2: cover area with bandage\nPrecaution_3: \nPrecaution_4: use ice to compress itching'),
 Document(metadata={'source': 's3_data/symptom_precaution.csv', 'row': 3}, page_content='Disease: Hypothyroidism\nPrecaution_1: reduce stress\nPrecaution_2: exercise\nPrecaution_3: eat healthy\nPrecaution_4: get proper sleep'),
 Document(metadata={'source': 's3_data/sympt

### Generating Embeddings


In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
import os


/Users/anuj/Desktop/GenAI/rag/rag_pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1. make constructer and define model_name, model  and run _load_model()
#### 2. define _load_model() -> here we define sentenceTransformer to model variable
#### 3. make method to generate embedding where param will be just texts(documents that split) then encode this split 

In [6]:

class EmbeddingManager:
    """Handles document embedding generation using sentenceTransformer"""
    def __init__(self,model_name:str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args: model_name: Huggingface model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the sentenceTransformer model"""
        try:
            print(f"loading Embedding Model:{self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded Successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"error loading model{self.model_name}: {e}")

    def generate_embedding(self,texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        Args:
            texts: List of text strings to embed
        
        returns:
            Numpy array of embeddings with shape(len(texts),embedding_dim)
        """

        if not self.model:
            raise ValueError("Model not loaded")
        print(f"Generating embedding for {len(texts)} texts...")
        embedding = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape:{embedding.shape}")
        return embedding

# Initialize the embedding Manager
embedding_manager = EmbeddingManager()
embedding_manager


loading Embedding Model:all-MiniLM-L6-v2
Model loaded Successfully. Embedding dimension: 384


### VectorStore

#### 1. make constructer and define collection_name, persist_directory which is directory(where vector will be stored), make two variable client,collection  and run _initialize_store()
#### 2. _initialize_store will be used to make chromadb client using PersistentClient from chromadb library then create or get collection.
#### 3. Now make a method to add documents to store in vectordb
##### 3(i). start with compare length of embedding and chunks
##### 3(ii). Zipping chunks and embedding to get metadata, page content, also create uuid for each one.
##### 3(iii). Now in the same function, we add metadata, page content, ids to collection



In [7]:
class VectorStore:
    """Manages document in a ChromaDB vector store"""
    def __init__(self,collection_name: str = 'Csv_data',persist_directory: str = "vector_store/"):
        """
        Initialize the vector store

            Args: 
                collection_name: Name of the chromadb collection
                persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB Client and collection"""
        try:
            # Create ChromaDB client
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            # Get or create Collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description":"Csv Data embedding for RAG"}
            )
            print(f"vector store initiallized. Collection:{self.collection_name}")
            print(f"Existing documents in collection:{self.collection.count()}")
            print("+++++++++++++++++++",self.client)
        except Exception as e:
            print(f"Error initializing vector store:{e}")
            raise

    def _upload_to_s3():
        pass
    
    def add_documents(self,documents: List[Any], embeddings:np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of Langchain documents
            embeddings: corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match of embeddings")
        print(f"adding {len(documents)} documents to vector store...")

        ids =[]
        metadatas = []
        document_text = []
        embedding_list = []

        for i , (doc,embeddings) in enumerate(zip(documents,embeddings)):
            # Create unique ids
            doc_id = uuid.uuid4().hex[:8]
            ids.append(doc_id)

            # Prepare Metadata
            metadata = dict(doc.metadata)
            metadata['doc_index']= i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            document_text.append(doc.page_content)

            # Embeddings
            embedding_list.append(embeddings.tolist())
        
        try:
            self.collection.add(
                ids = ids,
                embeddings = embedding_list,
                documents=document_text,
                metadatas=metadatas
            )
            print(f"succesfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection:{self.collection.count()}")
        except Exception as e:
            print(f"error adding documents to vector store {e}")
            raise

vectorstore = VectorStore()
vectorstore


vector store initiallized. Collection:Csv_data
Existing documents in collection:0
+++++++++++++++++++ <chromadb.api.client.Client object at 0x1668a3a90>


In [8]:
# Convert The text to embedding
texts = [doc.page_content for doc in chunks]

embeddings = embedding_manager.generate_embedding(texts)

# Store in vectorDB
vectorstore.add_documents(chunks,embeddings)


Generating embedding for 5135 texts...


Batches: 100%|██████████| 161/161 [00:14<00:00, 10.78it/s]


Generated embeddings with shape:(5135, 384)
adding 5135 documents to vector store...
succesfully added 5135 documents to vector store
Total documents in collection:5135


### Retriever Pipeline from VectorStore

In [9]:
class RAGRetriever:
    """Handles query-based retrieval from the vector Store"""
    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """

        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
    
    def retrieve(self,query:str,top_k: int = 5, score_threshold:float = 0.0)-> List[dict[str,Any ]]:
        """
            Retrieve relevant documents for a query

            Args:
                query: The search query
                top_k: Number of top results  to return
                score_threshold: Minimum similarity score threshold
            
            Returns:
                List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top k: {top_k}, score threshold: {score_threshold}")

        # generate query embedding
        query_embedding = self.embedding_manager.generate_embedding([query])[0]     # [query] -> convert full string into list

        # search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings= [query_embedding.tolist()],
                n_results= top_k
            )

            # Process results
            retrieved_docs = []
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i , (doc_id,document,metadata,distance) in enumerate(zip(ids,documents,metadatas,distances)):
                    similarity_score = 1-distance
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content':document,
                            'metadata':metadata,
                            'distance':distance,
                            'similarity_score':similarity_score,
                            'distance': distance,
                            'rank':i+1
                        })
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            return retrieved_docs
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever = RAGRetriever(vectorstore,embedding_manager)



In [10]:
rag_retriever.retrieve("Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated. High blood pressure typically does not cause symptoms")

Retrieving documents for query: 'Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated. High blood pressure typically does not cause symptoms'
Top k: 5, score threshold: 0.0
Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 10.92it/s]

Generated embeddings with shape:(1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'faafe38b',
  'content': 'Disease: Hypertension\nDescription: Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated. High blood pressure typically does not cause symptoms.',
  'metadata': {'doc_index': 5108,
   'source': 's3_data/symptom_Description.csv',
   'row': 14,
   'content_length': 259},
  'distance': 0.05761760473251343,
  'similarity_score': 0.9423823952674866,
  'rank': 1}]

#### Integration vectordb context pipeline with LLM output

In [11]:
### Simple RAG pipeline with groq LLM
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY =  os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash",temperature=0.8, api_key = GOOGLE_API_KEY)

def rag(query,retriever,llm,top_k=5,min_score=0.2,return_context=False):
    """
        RAG pipeline with extra features:
        -   Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query,top_k=top_k,score_threshold= min_score)
    if not results:
        return {'answer': 'No relevant context found.','sources':[],'confidence':0.0,'context':''}

    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'sources': doc['metadata'].get('source_file',doc['metadata'].get('source','unknown')),
        'page': doc['metadata'].get('page','unknow'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]

    confidence = max([doc['similarity_score'] for doc in results])

    # generater answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion:{query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context,query=query)])

    output = {
        'answer': response.content,
        'sources':sources,
        'confidence':confidence
    }
    if return_context:
        output['context'] = context
    return output

# example usage:
result = rag("what are the cause of Hypothyroidism?", rag_retriever,llm,top_k=3,min_score=0.1,return_context=True)
print("Answer:",result['answer'])
print("Sources:",result['sources'])
print("confidence:",result['confidence'])
print("Context Preview:",result['context'][:300])



Retrieving documents for query: 'what are the cause of Hypothyroidism?'
Top k: 3, score threshold: 0.1
Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.08it/s]

Generated embeddings with shape:(1, 384)
Retrieved 3 documents (after filtering)


Answer: The provided context lists symptoms of Hypothyroidism, but does not state its causes.
Sources: [{'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score': 0.18528175354003906, 'preview': 'Disease: Hypothyroidism\nSymptom_1: fatigue\nSymptom_2: weight_gain\nSymptom_3: mood_swings\nSymptom_4: lethargy\nSymptom_5: dizziness\nSymptom_6: puffy_face_and_eyes\nSymptom_7: enlarged_thyroid\nSymptom_8: brittle_nails\nSymptom_9: swollen_extremeties\nSymptom_10: depression\nSymptom_11: irritability\nSymptom...'}, {'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score': 0.18528175354003906, 'preview': 'Disease: Hypothyroidism\nSymptom_1: fatigue\nSymptom_2: weight_gain\nSymptom_3: mood_swings\nSymptom_4: lethargy\nSymptom_5: dizziness\nSymptom_6: puffy_face_and_eyes\nSymptom_7: enlarged_thyroid\nSymptom_8: brittle_nails\nSymptom_9: swollen_extremeties\nSymptom_10: depression\nSymptom_11: irritability\nSymptom...'}, {'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score': 0.1852

In [12]:
print(result)

{'answer': 'The provided context lists symptoms of Hypothyroidism, but does not state its causes.', 'sources': [{'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score': 0.18528175354003906, 'preview': 'Disease: Hypothyroidism\nSymptom_1: fatigue\nSymptom_2: weight_gain\nSymptom_3: mood_swings\nSymptom_4: lethargy\nSymptom_5: dizziness\nSymptom_6: puffy_face_and_eyes\nSymptom_7: enlarged_thyroid\nSymptom_8: brittle_nails\nSymptom_9: swollen_extremeties\nSymptom_10: depression\nSymptom_11: irritability\nSymptom...'}, {'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score': 0.18528175354003906, 'preview': 'Disease: Hypothyroidism\nSymptom_1: fatigue\nSymptom_2: weight_gain\nSymptom_3: mood_swings\nSymptom_4: lethargy\nSymptom_5: dizziness\nSymptom_6: puffy_face_and_eyes\nSymptom_7: enlarged_thyroid\nSymptom_8: brittle_nails\nSymptom_9: swollen_extremeties\nSymptom_10: depression\nSymptom_11: irritability\nSymptom...'}, {'sources': 's3_data/dataset.csv', 'page': 'unknow', 'score'

In [13]:
import boto3
import os

def data_ingestion_from_s3():
    s3 = boto3.client(
        's3',
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=os.getenv("AWS_REGION")
    )

    bucket_name = os.getenv("S3_BUCKET_NAME")
    prefix = "carequery/"

    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    local_files = os.getenv("S3_DOWNLOADED_FILES")
    os.makedirs(local_files,exist_ok=True )


    for obj in response.get("Contents", []):
        key = obj["Key"]
        if key.endswith(".csv"):
            filename = os.path.basename(key)
            local_csv_path = os.path.join(local_files,filename)
            if not os.path.exists(local_csv_path):
                s3_obj = s3.get_object(Bucket=bucket_name, Key=key)
                csv_content = s3_obj["Body"].read().decode("utf-8")
                # Write to a temporary file
                with open(local_csv_path,'w', encoding="utf-8") as f:
                    f.write(csv_content)
    return local_files


In [14]:
data_ingestion_from_s3()

's3_data/'

In [15]:
# ### Document Structure - DATA Ingestion
# import os
# from langchain_core.documents import Document
# ## Directory Loader - To read all files from a directory
# from langchain_community.document_loaders import DirectoryLoader
# from langchain_community.document_loaders import CSVLoader
# import os
# ## load all the text fies from the directory
# csv_files = data_ingestion_from_s3()
# dir_loader = DirectoryLoader(
    
#     csv_files,
#     glob = "**/*.csv", # Pattern to match files
#     loader_cls= CSVLoader, # loader class to use
#     loader_kwargs={'encoding':'utf-8'},
#     show_progress=False
# )

# documents =dir_loader.load()

# print(len(documents))
